# KM algorithmn

## Input
Input 1: matrix with column as each doctor, and patient as rownames        

Input 2: capacity

## Algorithm Explanation

The algorithm implemented below is the Hungarian Algorithm, also called KM. The basic concept is to create an mxn matrix where m is the number of available doctor spots (doctor 1 spot 1, doctor 1 spot 2, doctor 2 spot 1,…etc), and n is the number of patients who need to be assigned. The columns of the matrix are filled with the patient rankings that have been initially input, and if the matrix is not square additional dummy variables of all 0s are added as needed. The goal of the algorithm is to perform addition and subtraction to the rows and columns of this cost matrix in order to determine an optimal assignment for the resulting cost matrix —> which translates directly to an optimal assignment for the original cost matrix.

## Overview of Steps：

1. Make sure matrix is sqaure
2. Row and column reduction
3. Optimality test
4. Assignment

In [1]:
import numpy as np

In [2]:
input_matrix = []
i = 1
number_of_doctor = int(input("The number of available doctors: "))
capacity = int(input("The maximum patient capacity of each doctor: "))
p = int(input('The number of patients to be assigned: '))

print('\n')
for i in range(p):
    holding = []
    count = 1
    for b in range(number_of_doctor):
        x = int(input("Out of all the doctors, what would you rank doctor %d? " % count))
        holding.append(x)
        count+=1
    i = i + 1
    if i < p:
        print('\nNext patient please\n')
    #arr2 = list(np.repeat(arr2, c))
    input_matrix.append(holding)
print('\nThank You')

The number of available doctors: 3
The maximum patient capacity of each doctor: 3
The number of patients to be assigned: 6


Out of all the doctors, what would you rank doctor 1? 1
Out of all the doctors, what would you rank doctor 2? 3
Out of all the doctors, what would you rank doctor 3? 2

Next patient please

Out of all the doctors, what would you rank doctor 1? 1
Out of all the doctors, what would you rank doctor 2? 2
Out of all the doctors, what would you rank doctor 3? 3

Next patient please

Out of all the doctors, what would you rank doctor 1? 1
Out of all the doctors, what would you rank doctor 2? 2
Out of all the doctors, what would you rank doctor 3? 3

Next patient please

Out of all the doctors, what would you rank doctor 1? 3
Out of all the doctors, what would you rank doctor 2? 1
Out of all the doctors, what would you rank doctor 3? 2

Next patient please

Out of all the doctors, what would you rank doctor 1? 3
Out of all the doctors, what would you rank doctor 2? 1
Out

In [5]:
## 1. Make sure it's sqaure matrix#############################

# repeat columns as needed for doctor capacity and add dummy variables to make square matrix, 
# with colnames = each spot for doctor, rownames = patients

matrix = np.repeat(input_matrix,capacity,axis=1)

dum=0
while(matrix.shape[0] < matrix.shape[1]):
    dum+=1
    matrix=np.vstack([matrix,[0]*matrix.shape[1]])
matrix=matrix.transpose()
#print('sqaure matrix',matrix)
matrix_for_check = matrix.copy()



########################################2. row reduction and column reduction#####################

# subtract the minimum value in each row from its row and the minimum value in each column from its column 
# in order to make the smallest entry in each row and each column = 0

dim=matrix.shape[0]
cur_mat=matrix
for i in range(dim):
    cur_mat[i]=cur_mat[i]-np.min(cur_mat[i])
for i in range(dim):
    cur_mat[:,i]=cur_mat[:,i]-np.min(cur_mat[:,i])
#print('row_reduced',cur_mat)
############################################################################################

# function which tallies how many times the overall minimum of the matrix appears in each row and each column

def count_zero(cur_mat,matrix_minimum):
    num_zero_row=[]
    for i in range(cur_mat.shape[0]):
        row_zero = np.where(cur_mat[i].astype(int)==matrix_minimum)[0]
        num_zero_row.append(len(row_zero))

    num_zero_col=[]
    for i in range(cur_mat.shape[1]):
        col_zero = np.where(cur_mat[:,i].astype(int)==matrix_minimum)[0]
        num_zero_col.append(len(col_zero))

    return num_zero_col,num_zero_row

#print(cur_mat)



##################################################optimality test########################################### 

#Use th
#
#

def optimality_test(tmp_mat,line):
    row_delete=[]
    row_remain=list(np.arange(0,dim))
    col_remain=list(np.arange(0,dim))
    col_delete=[]
    matrix_minimum=np.min(tmp_mat)
    num_zero_col,num_zero_row = count_zero(tmp_mat,matrix_minimum)
    #print(num_zero_col,num_zero_row)
    result = False  
    while(1):
        if result == False:
            if np.max(num_zero_row) >= np.max(num_zero_col):
                #print("step ",line)
                tmp_row_delete=np.where(np.array(num_zero_row) == np.max(num_zero_row))[0][0]
                #print("delete row ", num_zero_row,tmp_row_delete)
                tmp_mat[tmp_row_delete]+=1
                #print('matrix',tmp_mat)
                row_delete.append(tmp_row_delete)
                row_remain.remove(tmp_row_delete)
                num_zero_col,num_zero_row = count_zero(tmp_mat,matrix_minimum)
                result=np.all((np.array(num_zero_row) == 0))+np.all((np.array(num_zero_col) == 0))
                #print(result == False)
            elif np.max(num_zero_col) > np.max(num_zero_row):
                #print("step ",line)
                tmp_col_delete=np.where(np.array(num_zero_col) == np.max(num_zero_col))[0][0]
                #print("delete col",tmp_col_delete)
                tmp_mat[:,tmp_col_delete]+=1
                #print('matrix',tmp_mat)
                col_delete.append(tmp_col_delete)
                col_remain.remove(tmp_col_delete)
                num_zero_col,num_zero_row = count_zero(tmp_mat,matrix_minimum)
                result=np.all((np.array(num_zero_row) == 0))+np.all((np.array(num_zero_col) == 0))
                #print(result == False)
            line+=1
        else:
            print("break")
            break
    return tmp_mat,row_delete,col_delete,row_remain,col_remain,line


tmp_mat=cur_mat.copy()
line=1
tmp_mat,row_delete,col_delete,row_remain,col_remain,line = optimality_test(tmp_mat,line)

while(line < dim):
    #print("round",line)
    tmp_mat,row_delete,col_delete,row_remain,col_remain,line = optimality_test(tmp_mat,line)
    #print("run_optimality_test")
    min_value=np.min(tmp_mat)
    #print(min_value,row_delete,col_delete,row_remain,col_remain)
    for i in row_delete:
        for l in col_delete:
            cur_mat[i,l]+=min_value
    for x in row_remain:
        for y in col_remain:
            cur_mat[x,y]-=min_value
    tmp_mat=cur_mat.copy()

#print(cur_mat)

############################################################################################

break


In [6]:
#################################################### Assignment ###############################################
cur_mat=cur_mat[:,:-dum]
i=1
final_row_index=[]
final_col_index=[]

#while(np.min(cur_mat)==minimum):
count=0
for count in range(p):
    matrix_minimum=np.min(cur_mat)
    num_zero_col,num_zero_row=count_zero(cur_mat,matrix_minimum)
    #print(count)
    #print("num_zero_col & row",num_zero_col,num_zero_row)
    if 1 in num_zero_row:
        #print("step remove row1",i)
        tmp_row=np.where(np.array(num_zero_row) == 1)[0][0]
        tmp_col = np.where(cur_mat[tmp_row]==min(cur_mat[tmp_row]))[0][0]
        #tmp_col=np.where(cur_mat[tmp_row].astype(int)==0)[0][0]
        cur_mat[tmp_row] += 999
        cur_mat[:,tmp_col] += 999
        final_row_index.append(tmp_row)
        final_col_index.append(tmp_col)
        #print(cur_mat,"tmp_row",tmp_row,"tmp_col",tmp_col)
    elif 1 in num_zero_col:
        #print("step remove col1",i)
        tmp_col=np.where(np.array(num_zero_col) == 1)[0][0]
        tmp_row=np.where(cur_mat[:,tmp_col]==min(cur_mat[:,tmp_col]))[0][0]
        cur_mat[tmp_row] += 999
        cur_mat[:,tmp_col] += 999
        final_row_index.append(tmp_row)
        final_col_index.append(tmp_col) 
        #print(cur_mat,"tmp_row",tmp_row,"tmp_col",tmp_col)
    else:
        min_col=np.min(list(filter(lambda x: x != 0, num_zero_col)))
        min_row=np.min(list(filter(lambda x: x != 0, num_zero_row)))
        if min_row <= min_col:
            #print("step remove row2",i)
            tmp_row=np.where(np.array(num_zero_row) == min_row)[0][0]
            tmp_col=np.where(cur_mat[tmp_row]==min(cur_mat[tmp_row]))[0][0]
            cur_mat[tmp_row] += 999
            cur_mat[:,tmp_col] += 999
            final_row_index.append(tmp_row)
            final_col_index.append(tmp_col)
            #print(cur_mat,"tmp_row",tmp_row,"tmp_col",tmp_col)  
        else:
            #print("step remove col2",i)
            tmp_col=np.where(np.array(num_zero_col) == min_col)[0][0]
            tmp_row=np.where(cur_mat[:,tmp_col]==min(cur_mat[:,tmp_col]))[0][0]
            cur_mat[tmp_row] += 999
            cur_mat[:,tmp_col] += 999
            final_row_index.append(tmp_row)
            final_col_index.append(tmp_col)
            #print(cur_mat,"tmp_row",tmp_row,"tmp_col",tmp_col)
            
    i+=1
    count+=1

        
            
#print(cur_mat, final_row_index,final_col_index)

############################################################################################



#########################################OUTPUT###################################################
start=0
#print(final_row_index,final_col_index)

for i in range(number_of_doctor):
    end= start+capacity
    l = list(range(start, end))
    start=end
    patient_index = np.array(np.where(np.isin(final_row_index,l))[0])
    patient_num=np.array(final_col_index)[np.array(patient_index.astype(int))]
    print("doctor "+ str(i+1)+" has the following patients: "+ str(patient_num+1))

############################################################################################    


doctor 1 has the following patients: [1 2 3]
doctor 2 has the following patients: [4 5]
doctor 3 has the following patients: [6]


# Use the scipy package in order to check the algorithmn

In [28]:
import numpy as np
from scipy.optimize import linear_sum_assignment

"""
# input list for patient, we have 3 doctors, 5 patients in this case
input_matrix = [[1,2,3,4],
                [2,4,1,3],
                [1,2,4,3],
                [3,1,2,4],
                [4,1,3,2]
               ]

matrix= np.repeat(input_matrix,2,axis=1)

#input: capacity for each doctor
capacity = 2

#
number_of_doctor=np.array(input_matrix).shape[1]

"""

# add dummy variables to make square matrix, with colnames = each spot for doctor, rownames = each spot for doctor

#print(matrix)
# KM algorithmn
row, col = linear_sum_assignment(matrix_for_check)

#best point in the matrix, minimized result
#print(row,col)

start=0
for i in range(number_of_doctor):
    end= start+capacity
    l = list(range(start, end))
    start=end
    patient_index = np.array(np.where(np.isin(row,l))[0])
    patient_num=np.array(col)[np.array(patient_index.astype(int))]
    #patient_num=list((col[:, None] == l).argmax(axis=0))
    #print(l,patient_num)
    print("doctor "+ str(i+1)+" has patient ",np.array(patient_num)+1)


[0 1 2 3 4 5 6 7 8 9] [6 4 2 3 0 8 9 7 5 1]
doctor 1 has patient  [7 5]
doctor 2 has patient  [3 4]
doctor 3 has patient  [1 9]
doctor 4 has patient  [10  8]
doctor 5 has patient  [6 2]


In [31]:
print(matrix_for_check) #each row is a spot for each doctor, patient is the column

[[2 4 2 2 2 3 2 0 0 0]
 [2 4 2 2 2 3 2 0 0 0]
 [1 5 1 1 1 2 1 0 0 0]
 [1 5 1 1 1 2 1 0 0 0]
 [3 3 3 3 5 5 3 0 0 0]
 [3 3 3 3 5 5 3 0 0 0]
 [4 2 5 4 4 4 4 0 0 0]
 [4 2 5 4 4 4 4 0 0 0]
 [5 1 4 5 3 1 5 0 0 0]
 [5 1 4 5 3 1 5 0 0 0]]


### References：

https://www.youtube.com/watch?v=Hbnpn_YM3YA


https://brilliant.org/wiki/hungarian-matching/
